In [ ]:
#导入库
import time #用于增加等待时间，以免爬取速度过快
from DrissionPage import WebPage #使用dp控制浏览器模拟访问
from DrissionPage._units.clicker import NoRectError #处理报错
from datetime import datetime #处理日期和时间
import pandas as pd #将爬取数据保存为 DataFrame 格式

#定义格式化时间函数：将毫秒级时间戳转换为可读的日期时间格式
def format_timestamp(timestamp_ms):

    # 将毫秒转换为秒
    timestamp_s = timestamp_ms / 1000
    # 转换为datetime对象
    dt = datetime.fromtimestamp(timestamp_s)
    # 格式化输出
    return dt.strftime('%Y-%m-%d %H:%M:%S')

# 获取当前时间，用于获取24小时以内的帖子
now = datetime.now()  

# 定义关键词列表
keywords = ["诈骗", "电信诈骗", "网络诈骗", "留学诈骗", "杀猪盘"]

# 定义空列表
info = []  # 用于暂时存储已爬取的笔记链接，不同的关键词可能有相同的笔记，防止重复爬取
data_list = []  # 用于存储最终要爬取笔记的详细信息

# 打开小红书(要打开至全屏，登陆后自动保持登陆状态)
wp = WebPage()  
wp.get('https://www.xiaohongshu.com/')

# 遍历每个关键词
for i, keyword in enumerate(keywords):
    
    # 如果不是第一个关键词，清空搜索框再输入关键词
    if i > 0: 
        close = wp.ele('.close-icon')  # 定位清空按钮
        close.click()  # 点击清空按钮
        time.sleep(1)  # 等待清空完成
        
    # 输入关键词并进行搜索
    inp = wp.ele('.search-input')  # 定位到搜索框
    inp.input(keyword)  # 输入关键词
    search = wp.ele('.search-icon')  # 定位搜索按钮
    search.click()  # 点击搜索
    time.sleep(3)  # 等待搜索结果加载

    # 如果是第一个关键词，点击最新按钮
    if i == 0:
        wp.ele(".filter").click() # 点击筛选器
        time.sleep(1)
        new = wp.ele(".dropdown-container", index=3).ele(".dropdown-items").ele("@text():最新") # 定位最新按钮
        new.click() #点击最新按钮
        time.sleep(1) # 等待搜索结果加载

    # 开始监听API 请求
    wp.listen.start(['web/v1/search/notes', 'api/sns/web/v1/feed', 'api/sns/web/v1/comment/page'])

    # 爬取数据
    for page in range(5):  # 设置滚动加载xx页
        note_elements = wp.eles('xpath://a[@class="cover ld mask"]')  # 获取所有笔记定位
        for j in range(len(note_elements)): #遍历每个笔记定位
            temp = note_elements[j]
            note_url = temp.attr('href') #获取浮窗笔记链接

            if note_url not in info:  # 避免重复爬取
                info.append(note_url)

                # 清除之前的监听数据
                wp.listen.clear()

                try:

                    # 点击笔记并等待响应
                    temp.click()

                    # 设置较短的超时时间
                    pack = wp.listen.wait(timeout=6)
                    if pack and pack.response and pack.response.body:
                        # 通过监听请求获取笔记内容
                        data = pack.response.body.get('data', {})
                        items = data.get('items', [{}])[0]
                        note_card = items.get('note_card', {})

                        # 获取并格式化时间
                        create_time = format_timestamp(note_card.get('time', 0))
                        update_time = format_timestamp(note_card.get('last_update_time', 0))
                        create_time_obj = datetime.strptime(create_time, '%Y-%m-%d %H:%M:%S')  # 转换为 datetime 对象

                        # 判断笔记是否在过去24小时内
                        if (now - create_time_obj).total_seconds() <= 86400:  # 86400秒 = 24小时

                            # 获取笔记标题
                            note_title = note_card.get('title', '')

                            # 获取用户名
                            username = note_card.get('user', {}).get('nickname', '')

                            # 获取图片链接列表
                            image_list = note_card.get('image_list', [])
                            image_urls = [image.get('url_default', '') for image in image_list]

                            # 获取原始显示的所有评论
                            comments = wp.eles('xpath://div[@class="content"]')
                            all_comments = [comment.text for comment in comments]
                            all_comments_text = '\n'.join(all_comments)

                        
                            # 将详细数据保存到列表中
                            data_list.append({
                                'url': note_url,
                                'title': note_title,
                                'username': username,
                                'post_time': create_time,
                                'update_time': update_time,
                                'post_content': note_card.get('desc', ''),
                                'replies': all_comments_text,
                                'image_url': '; '.join(image_urls),  # 将多个图片链接用分号隔开
                                'keyword': keyword  # 保存对应关键词
                            })
                except NoRectError as e:
                    print(f"发生 NoRectError 错误: {e}，跳过当前笔记：{note_url}")
                    continue

                except Exception as e: #避免程序因单个错误中断运行
                    print(f"发生错误: {e}")
                    continue

                # 关闭笔记
                close_button = wp.ele('xpath://div[@class="close close-mask-dark"]')
                if close_button:
                    close_button.click()
                time.sleep(1)

            # 确保只爬取当前页面数据，跳出循环
            if j == len(note_elements) - 1:
                break

# 将数据转换为 DataFrame 
df = pd.DataFrame(data_list)

# 获取当前日期并生成文件名
current_date = datetime.now().strftime('%Y-%m-%d')  
output_file_name = f"{current_date}.csv"  

# 导出到 CSV 文件
df.to_csv(output_file_name, index=False, encoding='utf-8-sig') 
print("小辣鸡成功啦")


发生 NoRectError 错误: 该元素没有位置及大小。，跳过当前笔记：https://www.xiaohongshu.com/search_result/6743eddf000000000702511f?xsec_token=ABpvBK2nQgCNmGc8Ua49Em0A2B4xyQQBEYoJF5vCv147U=&xsec_source=pc_search
发生 NoRectError 错误: 该元素没有位置及大小。，跳过当前笔记：https://www.xiaohongshu.com/search_result/6743df5c000000000203a698?xsec_token=ABpvBK2nQgCNmGc8Ua49Em0K3qC0mNWC9mnBUJxCuJtJk=&xsec_source=pc_search
发生 NoRectError 错误: 该元素没有位置及大小。，跳过当前笔记：https://www.xiaohongshu.com/search_result/6745172500000000020293fa?xsec_token=ABJkUXlQwYeOkSAtYOUfOGLnOVbb3eBkxuLqfk2yHYAaM=&xsec_source=pc_search
小辣鸡成功啦
